In [5]:
# import dependencies
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import image
from keras.applications.vgg19 import (
    VGG19,
    preprocess_input,
    decode_predictions
)

In [17]:
names = ['with_mask', 'without_mask']
batch = 32

In [15]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        validation_split = 0.2)
train_datagen

In [19]:
train_generator = train_datagen.flow_from_directory(
    directory = 'Data/',
    target_size = (220,220),
    batch_size = batch,
    class_mode = 'categorical',
    subset = 'training')

Found 8834 images belonging to 2 classes.


In [20]:
test_generator = train_datagen.flow_from_directory(
    directory = 'Data/',
    target_size = (220,220),
    batch_size = batch,
    class_mode = 'categorical',
    subset = 'validation')

Found 2208 images belonging to 2 classes.
